In [ ]:
#################################################################
#Bring News From NewsAPI
#################################################################


import requests
import pandas as pd

# Your NewsAPI key
API_KEY = "ENTER THE API KEY HERE"

# Base URL for fetching news
BASE_URL = "https://newsapi.org/v2/everything"

# Set parameters for the search
language = input("What language do you want the news in? (Example: en for English, ar for Arabic, fr for French): ").strip()
query = input("Enter the keyword for news search ( Middle East): ").strip()
page_size = 100  # Number of news items per page
max_pages = 5  # Maximum number of pages

params = {
    'q': query,          # Keyword for fetching news
    'language': language,  # User-selected language
    'apiKey': API_KEY,   # API key for authentication
    'pageSize': page_size  # Number of news items per request
}


# List to store all news articles
all_articles = []

# Fetch news from multiple pages
for page in range(1, max_pages + 1):
    print(f"Fetching page {page} of news...")
    params['page'] = page
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        articles = data.get('articles', [])
        if not articles:
            print(f"No news found on page {page}.")
            break
        all_articles.extend(articles)
    else:
        print(f"Error fetching data from page {page}: {response.status_code} - {response.text}")
        break

# Check if data exists
if not all_articles:
    print("No matching news articles found.")
    exit()

# Convert news to DataFrame
news_data = pd.DataFrame(all_articles)

# Select only the required columns
columns_to_keep = ['source', 'author', 'title', 'description', 'url', 'publishedAt']
news_data = news_data[columns_to_keep]

# Extract source name
news_data['source'] = news_data['source'].apply(lambda x: x.get('name') if isinstance(x, dict) else x)

# Clean the data
news_data.dropna(subset=['title', 'description'], inplace=True)

# Save data to CSV file
output_file = f"news_{query.replace(' ', '_')}.csv"
news_data.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"News successfully saved to file: {output_file}")

# Download file in Google Colab







try:
    from google.colab import files
    files.download(output_file)
    print("File downloaded successfully. Check your device.")
except ImportError:
    print(f"To download the file, look for {output_file} in the current directory.")


What language do you want the news in? (Example: en for English, ar for Arabic, fr for French): ar
Enter the keyword for news search ( Middle East): الأردن
Fetching page 1 of news...
Fetching page 2 of news...
Error fetching data from page 2: 426 - {"status":"error","code":"maximumResultsReached","message":"You have requested too many results. Developer accounts are limited to a max of 100 results. You are trying to request results 100 to 200. Please upgrade to a paid plan if you need more results."}
News successfully saved to file: news_الأردن.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File downloaded successfully. Check your device.


In [5]:
#################################################################
#Spilit News
#################################################################


import os
from google.colab import files

# Upload the news file via Google Colab
uploaded = files.upload()  # Opens a window for manual file selection

# Get the uploaded file name
file_name = list(uploaded.keys())[0]

# Number of splits
num_splits = 5

# Read the data
news_data = pd.read_csv(file_name, encoding="utf-8-sig")

# Calculate the number of rows per spilit
rows_per_split = len(news_data) // num_splits

# Create a folder to save the spilit files
output_folder = "split_news"
os.makedirs(output_folder, exist_ok=True)

# Split the data and save each part as a separate file
for i in range(num_splits):
    start_idx = i * rows_per_split
    end_idx = (i + 1) * rows_per_split if i < num_splits - 1 else len(news_data)

    split_data = news_data.iloc[start_idx:end_idx]
    split_filename = os.path.join(output_folder, f"news_part_{i+1}.csv")
    split_data.to_csv(split_filename, index=False, encoding="utf-8-sig")

    print(f" Saved part {i+1} in: {split_filename}")

# Download the files directly via Google Colab
print(" Downloading files...")
for i in range(num_splits):
    files.download(os.path.join(output_folder, f"news_part_{i+1}.csv"))

print(" All files have been successfully downloaded!")


Saving news_Middle_East.csv to news_Middle_East (2).csv
 Saved part 1 in: split_news/news_part_1.csv
 Saved part 2 in: split_news/news_part_2.csv
 Saved part 3 in: split_news/news_part_3.csv
 Saved part 4 in: split_news/news_part_4.csv
 Saved part 5 in: split_news/news_part_5.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 All files have been successfully downloaded!


In [7]:
pip install dotenv

In [ ]:
#################################################################
#Creat Arabic Articel From News using OpenAi
#################################################################

import openai
import time
import logging
from tqdm import tqdm
from google.colab import files
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configure logging settings
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


# Manually set the API key (Replace with your actual OpenAI API key)
os.environ["OPENAI_API_KEY"] = "enter the keys here"

# Retrieve it securely in the code
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError(" OpenAI API Key is missing! Make sure to set it properly.")


# Initialize OpenAI client
client = openai.OpenAI(api_key=api_key)

# Function to upload file in Google Colab
def upload_file():
    """Allows the user to upload a CSV file in Google Colab."""
    try:
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0] if uploaded else None
        if not file_name:
            raise ValueError("No file uploaded.")
        return file_name
    except Exception as e:
        logging.error(f"Error while uploading file: {e}")
        exit()

# Function to read CSV data with multiple encoding attempts
def load_data(file_name):
    """Loads the CSV file using different encoding options to avoid read errors."""
    encodings = ['utf-8-sig', 'utf-8', 'ISO-8859-1']
    for enc in encodings:
        try:
            news_data = pd.read_csv(file_name, encoding=enc, engine='python')
            logging.info(f"Data loaded successfully using {enc} encoding!")
            return news_data
        except Exception:
            logging.warning(f"Failed to load using {enc}. Trying next encoding...")

    raise ValueError("Failed to read the file with all attempted encodings.")

# Function to generate an article using OpenAI GPT-4
def generate_article(title, description):
    """Generates an analytical article based on the given title and description using OpenAI's GPT-4 API."""

    # Handle missing values
    if pd.isna(title) or pd.isna(description):
        return None

    # Prompt for article generation
    prompt = f"""Title: {title}
Description: {description}

Write a **high-quality, professional, and in-depth analytical article in Arabic** on this topic. The article should be **at least 1000 words long** and structured in a **clear, logical, and engaging manner**.

Ensure that:
- The article is written in a **formal and journalistic style**, maintaining **academic depth and accuracy**.
- The content is **cohesive, well-organized, and not fragmented**, with a **strong introduction, well-developed body, and compelling conclusion**.
- The analysis is **supported by evidence, expert opinions, and real-world examples** to enhance credibility.
- The language is **rich, sophisticated, and free from redundancy**, while keeping the content engaging and informative.
- The article **avoids generic AI-style phrasing** and feels **authentic, well-researched, and human-like**.
- The topic is covered from **multiple angles**, addressing **historical context, current trends, and future implications**.
- The conclusion provides **valuable insights, recommendations, and a thought-provoking ending**.

Make sure the final article reads as if it was written by a **seasoned journalist or academic expert**.
"""

    max_tokens = 4096  # Define token limit
    max_retries = 5  # Increase retry attempts for robustness
    base_delay = 10  # Initial delay for retrying API requests

    for attempt in range(max_retries):
        try:
            # OpenAI API call for generating the article
            response = client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are a professional journalist writing in-depth analytical articles in Arabic."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=max_tokens,
                temperature=0.9
            )
            return response.choices[0].message.content.strip()

        except openai.RateLimitError:
            logging.warning(f"Rate limit exceeded. Retrying in {base_delay} seconds...")
            time.sleep(base_delay)
            base_delay *= 2

        except openai.OpenAIError as e:
            logging.error(f"OpenAI API error: {e}")
            time.sleep(base_delay)

        except Exception as e:
            logging.error(f"Unexpected error: {e}")
            time.sleep(base_delay)

    logging.error(" Failed to generate article after multiple attempts.")
    return None

# Main function to process data and generate articles
def main():
    """Main execution function to handle file upload, data processing, article generation, and file export."""

    # Upload the file
    file_name = upload_file()

    # Load the dataset
    news_data = load_data(file_name)

    # Validate required columns
    required_columns = ['title', 'description']
    missing_columns = [col for col in required_columns if col not in news_data.columns]
    if missing_columns:
        raise ValueError(f"Missing columns: {missing_columns}")

    articles = []

    # Iterate through the dataset and generate articles
    for _, row in tqdm(news_data.iterrows(), total=len(news_data), desc="Generating articles"):
        article = generate_article(row.get('title', ''), row.get('description', ''))
        if article:
            row_dict = row.to_dict()
            row_dict['article'] = article
            articles.append(row_dict)

    # Save the generated articles to a CSV file
    output_file = 'generated_articles.csv'
    pd.DataFrame(articles).to_csv(output_file, index=False, encoding='utf-8-sig')

    # Download the output file
    files.download(output_file)
    logging.info(" Download successful.")

# Run the main program
if __name__ == "__main__":
    main()


Saving news_part_3 (1).csv to news_part_3 (1).csv


Generating articles: 100%|██████████| 20/20 [13:29<00:00, 40.48s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#################################################################
#Creat English Articel
#################################################################

import openai
import pandas as pd
import os
import time
import logging
from tqdm import tqdm
from google.colab import files

# Logging configuration
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ✅ Set OpenAI API key (ensure it's securely managed)
client = openai.OpenAI(api_key="enter the keys here")  

def upload_file():
    try:
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0] if uploaded else None
        if not file_name:
            raise ValueError("No file uploaded.")
        return file_name
    except Exception as e:
        logging.error(f"Error while uploading file: {e}")
        exit()

# Read CSV data with encoding handling
def load_data(file_name):
    encodings = ['utf-8-sig', 'utf-8', 'ISO-8859-1']
    for enc in encodings:
        try:
            news_data = pd.read_csv(file_name, encoding=enc, engine='python')
            logging.info(f"Data loaded successfully using {enc} encoding!")
            return news_data
        except Exception:
            logging.warning(f"Failed to load using {enc}. Trying next encoding...")

    raise ValueError("Failed to read the file with all attempted encodings.")

# Generate an article using GPT-4
def generate_article(title, description):
    if pd.isna(title) or pd.isna(description):
        return None

    prompt = f"""Title: {title}
    Description: {description}

    Write a professional analytical article in English that explores this topic in depth.
    Ensure that:
- The article is written in a **formal and journalistic style**, maintaining **academic depth and accuracy**.
- The content is **cohesive, well-organized, and not fragmented**, with a **strong introduction, well-developed body, and compelling conclusion**.
- The analysis is **supported by evidence, expert opinions, and real-world examples** to enhance credibility.
- The language is **rich, sophisticated, and free from redundancy**, while keeping the content engaging and informative.
- The article **avoids generic AI-style phrasing** and feels **authentic, well-researched, and human-like**.
- The topic is covered from **multiple angles**, addressing **historical context, current trends, and future implications**.
- The conclusion provides **valuable insights, recommendations, and a thought-provoking ending**.

Make sure the final article reads as if it was written by a **seasoned journalist or academic expert**.
"""

    max_tokens = 4096
    max_retries = 3
    retry_delay = 10

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are a professional journalist who writes in-depth analytical articles in English."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=max_tokens,
                temperature=0.9
            )
            return response.choices[0].message.content.strip()
        except openai.RateLimitError:
            logging.warning(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        except Exception as e:
            logging.error(f"Unexpected error: {e}")
            time.sleep(retry_delay)

    logging.error("Failed to generate article after multiple attempts.")
    return None

# Main function to process data and generate articles
def main():
    file_name = upload_file()
    news_data = load_data(file_name)

    required_columns = ['title', 'description']
    missing_columns = [col for col in required_columns if col not in news_data.columns]
    if missing_columns:
        raise ValueError(f"Missing columns: {missing_columns}")

    articles = []
    for _, row in tqdm(news_data.iterrows(), total=len(news_data), desc="Generating articles"):
        article = generate_article(row.get('title', ''), row.get('description', ''))
        if article:
            row_dict = row.to_dict()
            row_dict['article'] = article
            articles.append(row_dict)

    # Save results to CSV file
    output_file = 'generated_articles.csv'
    pd.DataFrame(articles).to_csv(output_file, index=False, encoding='utf-8-sig')

    # Download the file
    files.download(output_file)
    logging.info("Download successful.")

# Run the main program
if __name__ == "__main__":
    main()


Saving news_part_2 (1).csv to news_part_2 (1).csv


Generating articles: 100%|██████████| 50/50 [16:55<00:00, 20.31s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#################################################################
#Creat Germany Articel
#################################################################

import openai
import pandas as pd
import os
import time
import logging
from tqdm import tqdm
from google.colab import files

# Logging configuration
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Set OpenAI API key (ensure it's securely managed)
client = openai.OpenAI(api_key="enter the keys here")  # 🔥 Replace with your actual API key

def upload_file():
    try:
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0] if uploaded else None
        if not file_name:
            raise ValueError("No file uploaded.")
        return file_name
    except Exception as e:
        logging.error(f"Error while uploading file: {e}")
        exit()

# Read CSV data with encoding handling
def load_data(file_name):
    encodings = ['utf-8-sig', 'utf-8', 'ISO-8859-1']
    for enc in encodings:
        try:
            news_data = pd.read_csv(file_name, encoding=enc, engine='python')
            logging.info(f"Data loaded successfully using {enc} encoding!")
            return news_data
        except Exception:
            logging.warning(f"Failed to load using {enc}. Trying next encoding...")

    raise ValueError("Failed to read the file with all attempted encodings.")

# Generate an article using GPT-4
def generate_article(title, description):
    if pd.isna(title) or pd.isna(description):
        return None

    prompt = f"""Title: {title}
    Description: {description}

    Write a professional analytical article in German that explores this topic in depth.
Ensure that:
- The article is written in a **formal and journalistic style**, maintaining **academic depth and accuracy**.
- The content is **cohesive, well-organized, and not fragmented**, with a **strong introduction, well-developed body, and compelling conclusion**.
- The analysis is **supported by evidence, expert opinions, and real-world examples** to enhance credibility.
- The language is **rich, sophisticated, and free from redundancy**, while keeping the content engaging and informative.
- The article **avoids generic AI-style phrasing** and feels **authentic, well-researched, and human-like**.
- The topic is covered from **multiple angles**, addressing **historical context, current trends, and future implications**.
- The conclusion provides **valuable insights, recommendations, and a thought-provoking ending**.

Make sure the final article reads as if it was written by a **seasoned journalist or academic expert**.
    """

    max_tokens = 4096
    max_retries = 3
    retry_delay = 10

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are a professional journalist who writes in-depth analytical articles in German."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=max_tokens,
                temperature=0.9
            )
            return response.choices[0].message.content.strip()
        except openai.RateLimitError:
            logging.warning(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        except Exception as e:
            logging.error(f"Unexpected error: {e}")
            time.sleep(retry_delay)

    logging.error("Failed to generate article after multiple attempts.")
    return None

# Main function to process data and generate articles
def main():
    file_name = upload_file()
    news_data = load_data(file_name)

    required_columns = ['title', 'description']
    missing_columns = [col for col in required_columns if col not in news_data.columns]
    if missing_columns:
        raise ValueError(f"Missing columns: {missing_columns}")

    articles = []
    for _, row in tqdm(news_data.iterrows(), total=len(news_data), desc="Generating articles"):
        article = generate_article(row.get('title', ''), row.get('description', ''))
        if article:
            row_dict = row.to_dict()
            row_dict['article'] = article
            articles.append(row_dict)

    # Save results to CSV file
    output_file = 'generated_articles.csv'
    pd.DataFrame(articles).to_csv(output_file, index=False, encoding='utf-8-sig')

    # Download the file
    files.download(output_file)
    logging.info("Download successful.")

# Run the main program
if __name__ == "__main__":
    main()


Saving articles_with_pexels_images (1).csv to articles_with_pexels_images (1).csv


Generating articles: 100%|██████████| 19/19 [10:41<00:00, 33.77s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#################################################################
#Bring Pics for Articel from Pexels
#################################################################


import pandas as pd
import requests
from google.colab import files

# Upload the CSV file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

# Pexels API key
PEXELS_API_KEY = "enter the keys here"

# Function to get an image URL from Pexels
def get_pexels_image(query):
    url = f"https://api.pexels.com/v1/search?query={query}&per_page=1"
    headers = {"Authorization": PEXELS_API_KEY}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data["photos"]:
            return data["photos"][0]["src"]["medium"]  # Can adjust the size if needed
    return None

# Apply the function to article titles
df["image_url"] = df["title"].apply(get_pexels_image)

# Save the updated file
output_file = "articles_with_pexels_images.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")

# Download the updated file
files.download(output_file)


Saving clean_articles_with_pexels_images (6).csv to clean_articles_with_pexels_images (6) (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
#################################################################
#Cleaing Articels from sym (#+*)
#################################################################
import pandas as pd

from google.colab import files

# Upload the CSV file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

# Define characters to remove
chars_to_remove = "#+*"

# Remove specified characters from all text columns
df = df.applymap(lambda x: ''.join(c for c in x if c not in chars_to_remove) if isinstance(x, str) else x)

# Save the cleaned file
output_file = "clean_articles_with_pexels_images.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")

# Download the updated file
files.download(output_file)


Saving articles_with_pexels_images (2) - articles_with_pexels_images (2).csv (1).csv to articles_with_pexels_images (2) - articles_with_pexels_images (2).csv (1).csv


<ipython-input-3-a71488ca9e1b>:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in x if c not in chars_to_remove) if isinstance(x, str) else x)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#################################################################
#Convert Articel from csv to word doc
#################################################################

!pip install python-docx  # Install the necessary library

import zipfile
from google.colab import files
from docx import Document

# Step 1: Upload the CSV file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Step 2: Read the CSV file
df = pd.read_csv(file_name)

# Step 3: Create a folder for Word documents
output_folder = "word_documents"
os.makedirs(output_folder, exist_ok=True)

# Step 4: Save each row as a separate Word file
for index, row in df.iterrows():
    doc = Document()

    # Use the first column as the title
    title_column = df.columns[0]
    doc.add_heading(str(row[title_column]), level=1)

    # Add the rest of the row data to the document
    for col in df.columns:
        doc.add_paragraph(f"{col}: {row[col]}")

    # Save the document
    output_file = os.path.join(output_folder, f"Article_{index+1}.docx")
    doc.save(output_file)

# Step 5: Zip all documents
zip_filename = "articles.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in os.listdir(output_folder):
        zipf.write(os.path.join(output_folder, file), file)

# Step 6: Download the ZIP file
files.download(zip_filename)


Saving clean_articles_with_pexels_images (2).csv to clean_articles_with_pexels_images (2) (2).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#################################################################
#Check the concotion with Database
#################################################################

import psycopg2

DATABASE_URL = "enter the keys here"

try:
    conn = psycopg2.connect(DATABASE_URL)
    cursor = conn.cursor()
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print(f"Connected to PostgreSQL. Version: {db_version}")

    cursor.close()
    conn.close()

except Exception as e:
    print(f"Database connection failed: {e}")


Connected to PostgreSQL. Version: ('PostgreSQL 16.7 (Debian 16.7-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [ ]:
#################################################################
#Send Articel to Database Sql
#################################################################

import psycopg2
import pandas as pd
from google.colab import files

# Upload CSV file manually
uploaded = files.upload()
CSV_FILE = list(uploaded.keys())[0]
df = pd.read_csv(CSV_FILE, encoding="utf-8-sig")

# Database connection URL
DATABASE_URL = "enter the keys here"

# Rename columns to match the database schema
df.rename(columns={"article": "content"}, inplace=True)

# Add default values for missing columns
df["category"] = df.get("category", "Uncategorized")
df["language"] = df.get("language", "en")

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(DATABASE_URL)
    cursor = conn.cursor()

    # Create the `articles` table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id SERIAL PRIMARY KEY,
            title TEXT NOT NULL,
            content TEXT NOT NULL,
            image TEXT,
            category TEXT DEFAULT 'Uncategorized',
            language TEXT DEFAULT 'en',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)

    conn.commit()  # Confirm table creation

    # Insert articles into the database
    for _, row in df.iterrows():
        try:
            cursor.execute(
                """
                INSERT INTO articles (title, content, image, category, language)
                VALUES (%s, %s, %s, %s, %s)
                """,
                (
                    row.get("title", "No Title"),
                    row.get("content", ""),
                    row.get("image_url", ""),
                    row.get("category", "Uncategorized"),
                    row.get("language", "en")
                )
            )
        except Exception as e:
            print(f" Error inserting article '{row.get('title', 'No Title')}': {e}")

    conn.commit()  # Save changes
    cursor.close()
    conn.close()

    print(" Articles successfully uploaded to the database!")

except Exception as e:
    print(f" Error connecting to the database: {e}")


Saving clean_articles_with_pexels_images (5).csv to clean_articles_with_pexels_images (5) (1).csv
 Articles successfully uploaded to the database!


In [ ]:
#################################################################
#Check the Number of Articels in Database
#################################################################

import psycopg2

DATABASE_URL = "enter the keys here"

try:
    conn = psycopg2.connect(DATABASE_URL)
    cursor = conn.cursor()

    cursor.execute("SELECT COUNT(*) FROM articles;")
    count = cursor.fetchone()[0]
    print(f"Total articles in database: {count}")

    cursor.close()
    conn.close()

except Exception as e:
    print(f"Error checking data: {e}")


Total articles in database: 38


In [ ]:
#################################################################
#Check the Conaction with Website API
#################################################################

import requests

response = requests.get("enter the keys here")
print(response.status_code)
print(response.json())


200
{'deploy': {'id': 'dep-cushmvdds78s739ppnb0'}}


In [ ]:
#################################################################
#Delete all Articels from Database
#################################################################

import psycopg2

# Database Configuration
DATABASE_URL = "enter the keys here"

try:
    conn = psycopg2.connect(DATABASE_URL)
    cursor = conn.cursor()

    # Delete all data from the articles table
    cursor.execute("DELETE FROM articles")
    conn.commit()

    print(" All articles have been deleted!")

    cursor.close()
    conn.close()

except Exception as e:
    print(f" Error: {e}")


 All articles have been deleted!
